In [ ]:
# import packages

import os
from operator import itemgetter    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import funcsigs
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder, LabelBinarizer
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV,KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
from funcsigs import signature
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from mlxtend.plotting import plot_learning_curves
from mlxtend.preprocessing import shuffle_arrays_unison

import tensorflow as tf
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical

import math
import seaborn as sns
from IPython.display import display

from statsmodels.formula import api
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [ ]:
# read in the data

df = pd.read_excel(r"dataset.xlsx")
print(df.head())


In [ ]:
# set target and features

target = 'COPD_Diagnose_ja_nein'
features = [i for i in df.columns if i not in [target]]

# make a copy of the original data

original_df = df.copy(deep=True)


In [ ]:
# show what are in the work space.

dir()


In [ ]:
# display some data information 

print(df.shape)


In [ ]:
# display some data information 

print(df.info())


In [ ]:
# display some data information 

print(df.describe())


In [ ]:
# check the number of values for all columns 

df.nunique().sort_values()


In [ ]:
# figure out numerical and categorical features

nu = df[features].nunique().sort_values()
nf = []; 
cf = []; 
nnf = 0; 
ncf = 0; 

for i in range(df[features].shape[1]):
    if nu.values[i]<=16: 
        cf.append(nu.index[i])
    else: nf.append(nu.index[i])

print('\n\033[1mInference:\033[0m The Datset has {} numerical and {} categorical features.'.format(len(nf),len(cf)))


In [ ]:
# set how many numerical features and categorical features.

Nnf = 13
Ncf = 6


In [ ]:
# count plot for categorical features

print('\033[1mVisualising Categorical Features:'.center(100))

fig, axs = plt.subplots(nrows = 2, ncols = 3)
ax = axs.flatten()
for i in range(len(cf)):
    if df[cf[i]].nunique()<=16:
        sns.countplot(x = cf[i], data = df, ax = ax[i])
        
plt.tight_layout()
plt.show()


In [ ]:
# distribution plot for numerical features

print('\033[1mNumeric Features Distribution'.center(130))

fig, axs = plt.subplots(nrows = 5, ncols = 3)
ax = axs.flatten()
for i in range(len(nf)):
    sns.distplot(df[nf[i]], ax = ax[i], hist_kws=dict(edgecolor="black", linewidth=2), bins=10)
plt.tight_layout()
ax[13].set_visible(False)
ax[14].set_visible(False)
plt.show()


In [ ]:
# boxplot for numerical features

fig, axs = plt.subplots(nrows = 5, ncols = 3)
ax = axs.flatten()
for i in range(len(nf)):
    df.boxplot(nf[i], ax = ax[i])
plt.tight_layout()
ax[13].set_visible(False)
ax[14].set_visible(False)
plt.show()


In [ ]:
# pairplots for all features 

# g = sns.pairplot(df)
# plt.title('Pairplots for all the Feature')
# g.map_upper(sns.kdeplot, levels=4, color=".2")
# plt.show()


In [ ]:
# compute the number of Null values in each feature

nvc = pd.DataFrame(df.isnull().sum().sort_values(), columns=['Total Null Values'])
nvc['Percentage'] = round(nvc['Total Null Values']/df.shape[0],3)*100
print(nvc)


In [ ]:
# review a few rows of the data

df.head()


In [ ]:
# replace the null values in a few features by the most frequent value in that column. 

df['Raceethnicity'].fillna(df['Raceethnicity'].mode()[0],inplace=True)
df['Sex'].fillna(df['Sex'].mode()[0],inplace=True)
df['Dentatestatus'].fillna(df['Dentatestatus'].mode()[0],inplace=True)


In [ ]:
# replace the null values in a some features by mean value.  

col_to_be_imputed = ['AgeatinterviewScreener', 'PovertyIncomeRatiounimputedincome', 'Pulseratebeatsminage5years','OverallaverageK1systolicBPage5', 'OverallaverageK5diastolicBPage5', 'Bodymassindex','Upperquadrantperiodontalassessments','Lowerquadrantperiodontalassessments','SumofpermanentDMFSduetodisease','SumofpermanentDFMSduetoanycause','SumofpermanentDFMSduetoanycause','SumofpermanentDFS','SumofpermanentDMFTduetodisease','SumofpermanentDMFTduetoanycause','Bleeding_Percentage','Furcation_SUM', 'MAL']
for item in col_to_be_imputed:
    df[item].fillna(df[item].mean(),inplace=True)


In [ ]:
# encoder categrical features by LabelEncoder. 

le = LabelEncoder()
df['Raceethnicity'] = le.fit_transform(df['Raceethnicity'])
df['Sex'] = le.fit_transform(df['Sex'])
df['Dentatestatus'] = le.fit_transform(df['Dentatestatus'])

# convert other features values from float to int

df['AgeatinterviewScreener'] = df['AgeatinterviewScreener'].astype(int)
df['PovertyIncomeRatiounimputedincome'] = df['PovertyIncomeRatiounimputedincome'].astype(int)
df['Pulseratebeatsminage5years'] = df['Pulseratebeatsminage5years'].astype(int)
df['OverallaverageK1systolicBPage5'] = df['OverallaverageK1systolicBPage5'].astype(int)
df['OverallaverageK5diastolicBPage5'] = df['OverallaverageK5diastolicBPage5'].astype(int)
df['Bodymassindex'] = df['Bodymassindex'].astype(int)
df['Upperquadrantperiodontalassessments'] = df['Upperquadrantperiodontalassessments'].astype(int)
df['Lowerquadrantperiodontalassessments'] = df['Lowerquadrantperiodontalassessments'].astype(int)
df['SumofpermanentDMFSduetodisease'] = df['SumofpermanentDMFSduetodisease'].astype(int)
df['SumofpermanentDFMSduetoanycause'] = df['SumofpermanentDFMSduetoanycause'].astype(int)
df['SumofpermanentDFMSduetoanycause'] = df['SumofpermanentDFMSduetoanycause'].astype(int)
df['SumofpermanentDFS'] = df['SumofpermanentDFS'].astype(int)
df['SumofpermanentDMFTduetodisease'] = df['SumofpermanentDMFTduetodisease'].astype(int)
df['SumofpermanentDMFTduetoanycause'] = df['SumofpermanentDMFTduetoanycause'].astype(int)
df['Bleeding_Percentage'] = df['Bleeding_Percentage'].astype(int)
df['Furcation_SUM'] = df['Furcation_SUM'].astype(int)
df['MAL'] = df['MAL'].astype(int)


In [ ]:
# display information of all features. 

df.info()


In [ ]:
# remove some useless features. 

df_feat = df.drop(['Raceethnicity','Sex','Dentatestatus','COPD_Diagnose_ja_nein'],axis=1)

# get labels for each sample.

target =  pd.get_dummies(df[['COPD_Diagnose_ja_nein']],drop_first=True)


In [ ]:
# get labels for each categrical feature. 

dummies = pd.get_dummies(df[['Raceethnicity','Sex','Dentatestatus', ]],drop_first=True)


In [ ]:
# display a few rows of the useful features. 

df_feat.head()


In [ ]:
# standardize all features data by sklearn StandardScaler

scaler = StandardScaler()
scaler.fit(df_feat)
df_scaled = scaler.transform(df_feat)
df_scaled = pd.DataFrame(df_scaled,columns=df_feat.columns[:19])
df_preprocessed = pd.concat([df_scaled,dummies,target], axis=1)
df_preprocessed.head()


In [ ]:
# Show number of features and number of samples

print('\n\033[1mInference:\033[0m The Datset consists of {} features & {} samples.'.format(df_preprocessed.shape[1], df_preprocessed.shape[0]))


In [ ]:
# see summarize of feature values

df_preprocessed.describe()


In [ ]:
# set y (values to be predicted), and X (feature values)

y = df_preprocessed['COPD_Diagnose_ja_nein']
X = df_preprocessed.drop('COPD_Diagnose_ja_nein', axis = 1)

print("y - Labels", y.shape)
print("X - N Label N id ", X.shape)
print(X.columns)


In [ ]:
# review the shapes of data frames

print(df_preprocessed.shape)
print(X.shape)
print(y.shape)


In [ ]:
# separate data to Tain or Test.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
print ('X_train: ', X_train.shape)
print ('X_test: ', X_test.shape)
print ('y_train: ', y_train.shape)
print ('y_test: ', y_test.shape)


In [ ]:
y.head()

In [ ]:
# build graph

X_T = X.transpose()
corr_A = X_T.corr()

def co(x):
    if x < 0: return 0 
    else: return x
    
corr_W = corr_A.applymap(co)


In [ ]:
# import packages

import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import networkx as nx
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy


In [ ]:
# set train and test

Y = y

perm = torch.randperm(len(Y))
train_mask = perm[:round(len(Y)*0.8)]
val_mask = perm[round(len(Y)*0.8):round(len(Y)*0.9)]
test_mask = perm[round(len(Y)*0.9):]
train_mask = train_mask.new_full((len(Y),), False, dtype=torch.bool)
train_mask[perm[:round(len(Y)*0.8)]] = True
val_mask = val_mask.new_full((len(Y),), False, dtype=torch.bool)
val_mask[perm[round(len(Y)*0.8):round(len(Y)*0.9)]] = True
test_mask = test_mask.new_full((len(Y),), False, dtype=torch.bool)
test_mask[perm[round(len(Y)*0.9):]] = True


In [ ]:
# prepare data and import packages

adj = corr_W.to_numpy()
sparse = scipy.sparse.coo_matrix(adj)
row = sparse.row
col = sparse.col
ww = sparse.data
edge_index = torch.stack([torch.LongTensor(row), torch.LongTensor(col)])
edge_weight = torch.FloatTensor(ww)


x = X.to_numpy()
x = torch.from_numpy(x)

y = Y.to_numpy()
y = torch.from_numpy(y)

from torch_geometric.data import Data

data = Data(x=x, edge_index=edge_index, edge_weight=edge_weight, y=y, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

data.x = data.x.float()
data.y = data.y.long()
data.edge_weight = data.edge_weight.float()


import time
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv


In [ ]:
# define a GNN class

class Net(torch.nn.Module):
    def __init__(self, hidden_channels, shared_weights=True, dropout=0.0):
        super().__init__()
        self.conv1 = GCNConv(19, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 2)
        self.dropout = dropout

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)

        return x.log_softmax(dim=-1)


In [ ]:
# set device and optimizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(hidden_channels=19, shared_weights=True, dropout=0.3).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), weight_decay=5e-4, lr=0.01)


In [ ]:
# define a train function

def train():
    model.train()
    optimizer.zero_grad()
#--    out = model(data.x, data.adj_t)
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)


In [ ]:
# define a test function

@torch.no_grad()
def test():
    model.eval()
#--    pred, accs = model(data.x, data.adj_t).argmax(dim=-1), []
    pred, accs = model(data).argmax(dim=-1), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs


In [ ]:
# run GNN and print results

best_val_acc = test_acc = 0
times = []
for epoch in range(1, 200):
    start = time.time()
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:04d}, Loss: {loss:.4f} Train: {train_acc:.4f}, '
          f'Val: {val_acc:.4f}, Test: {tmp_test_acc:.4f}, '
          f'Final Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")
